In [127]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
import pandas as pd

load_dotenv()

llm = ChatOpenAI(model="gpt-5",temperature=0)

In [128]:
def extract_dataframe_from_result(intermediate_steps):
    """
    Extract the dataframe result from agent intermediate steps.
    Returns the actual pandas DataFrame object when possible.
    """
    if not intermediate_steps:
        return None
    
    # Get the result from the last tool execution
    last_result = intermediate_steps[-1][1]
    
    # If it's already a DataFrame, return it
    if isinstance(last_result, pd.DataFrame):
        return last_result
    
    # If it's a string representation, try to extract the query and re-execute
    action = intermediate_steps[-1][0]
    if hasattr(action, 'tool_input') and 'query' in action.tool_input:
        query = action.tool_input['query']
        print(f"Executing query: {query}")
        try:
            result_df = eval(query.split('\n')[-1])
            if isinstance(result_df, pd.DataFrame):
                return result_df
        except:
            pass
    return last_result

In [142]:
def analysis_agent(path, llm, query):
    df = pd.read_csv(path)
    agent_executor = create_pandas_dataframe_agent(
        llm,
        df,
        agent_type="tool-calling",
        allow_dangerous_code=True,
        return_intermediate_steps=True  # Add this line
    )

    result = agent_executor.invoke({"input": query})
    
    # Check if intermediate_steps exists before trying to extract
    if "intermediate_steps" in result:
        df = extract_dataframe_from_result(result["intermediate_steps"])
    else:
        df = None
        print("No intermediate steps found in result")
    
    return result, df

In [143]:
response,df = analysis_agent("C:\\vscode\\csv_agent\\data\\data.csv", llm, "Display data where discount percentage is greater than 10%.")

Executing query: filtered = df[df['discount_pct'] > 0.10]
filtered_shape = filtered.shape
filtered_head = filtered.head(20)
filtered_shape, filtered_head


In [144]:
df

((1074, 19),
           date  year  quarter  month region     segment  channel  \
 1   2023-01-01  2023        1      1  North    Consumer   Retail   
 4   2023-01-01  2023        1      1  South         SMB   Retail   
 8   2023-01-01  2023        1      1  North  Enterprise   Retail   
 13  2023-01-02  2023        1      1  North         SMB   Online   
 16  2023-01-02  2023        1      1   West  Enterprise   Online   
 20  2023-01-03  2023        1      1  South    Consumer   Online   
 21  2023-01-03  2023        1      1  North  Enterprise   Online   
 30  2023-01-04  2023        1      1   West    Consumer   Online   
 37  2023-01-04  2023        1      1   West    Consumer   Retail   
 42  2023-01-05  2023        1      1   East         SMB   Online   
 47  2023-01-05  2023        1      1  South    Consumer   Online   
 52  2023-01-05  2023        1      1  South         SMB   Online   
 55  2023-01-05  2023        1      1   East    Consumer   Online   
 62  2023-01-06  2023

In [145]:
response['output']

'There are 1,074 rows where discount_pct > 0.10. Showing the first 20 rows:\n\n          date  year  quarter  month region     segment  channel         product_category    product_name      sku  units_sold  unit_price  discount_pct  gross_revenue      cogs  tax_pct  tax_amount  returned_units  net_revenue\n1   2023-01-01  2023        1      1  North    Consumer   Retail                Hardware       Delta Hub   H-8359           1    37377.22          0.15       31770.64  14901.88     0.12     3812.48               0     35583.11\n4   2023-01-01  2023        1      1  South         SMB   Retail                Hardware       Delta Hub   H-3803           1    14792.44          0.15       12573.57   7233.13     0.05      628.68               0     13202.25\n8   2023-01-01  2023        1      1  North  Enterprise   Retail  Professional Services     Integration  PS-2827           3     8168.76          0.15       20830.33   8443.69     0.05     1041.52               0     21871.85\n13  2023-